### Complete RAG Chain: OCI Embeddings, Oracle DB Vector Store, OCI Reraker and OCI GenAI

* ispired by: https://docs.llamaindex.ai/en/stable/examples/low_level/vector_store.html
* added **reranker** deployed ss OCI DS Model Deployment
* simplified using **factory methods**(create_xxx) from **prepare_chain** module

In [1]:
import logging

from typing import List, Any, Optional, Dict, Tuple
from llama_index.vector_stores.types import (
    VectorStoreQuery,
)
import llama_index
from llama_index import VectorStoreIndex, ServiceContext

import oci
import ads

# only to check versions!
import oracledb

from oci_utils import load_oci_config
from oracle_vector_db import OracleVectorStore

# using factory methods
from prepare_chain import create_reranker, create_embedding_model, create_llm

In [2]:
# version I'm using
print(f"oracledb version: {oracledb.__version__}")
print(f"oci version: {oci.__version__}")
print(f"llama_index version: {llama_index.__version__}")

oracledb version: 2.0.0.dev20231121
oci version: 2.112.1+preview.1.1649
llama_index version: 0.9.21


In [3]:
# parameters

# for similarity query
TOP_K = 10
# for Reranker
TOP_N = 4

In [4]:
# for debugging
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### The request for our RAG chain

In [5]:
question = "Write an e-mail, to a customer called Luigi, explaining what is JSON Relational Duality in Oracle Database 23c. Explain with details."

#### Embeddings

In [6]:
# setup
oci_config = load_oci_config()

# need to do this way
api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual

embed_model = create_embedding_model(auth=api_keys_config)

In [7]:
# embed the query using OCI GenAI
query_embedding = embed_model.embed_documents([question])[0]

#  wrap in llama-index
query_obj = VectorStoreQuery(query_embedding=query_embedding, similarity_top_k=TOP_K)

#### Oracle DB Vector Store

In [8]:
v_store = OracleVectorStore(verbose=False)

#### Documents retrieved using only Oracle Vector Store DB

In [9]:
%%time

q_result = v_store.query(query_obj)

CPU times: user 20.5 ms, sys: 5.68 ms, total: 26.2 ms
Wall time: 950 ms


In [10]:
for n, id, sim in zip(q_result.nodes, q_result.ids, q_result.similarities):
    print(f"Doc. id: {id}")
    # -sim, caus dot compute the negated dot product (see vector store docs)
    print(f"Similarity: {-sim}")
    print(n.text)
    print("")

Doc. id: bfbaa15634643a2aac6e77b85a4f8a335eba697d741aaaa3646bdfb07fa3ba22
Similarity: 0.69
2 Application Development JSON JSON-Relational Duality JSON Relational Duality Views are fully updatable JSON views over relational data. Data is still stored in relational tables in a highly efficient normalized format but can be accessed by applications in the form of JSON documents. Duality views provide you with game-changing flexibility and simplicity by overcoming the historical challenges developers have faced when building applications using relational or document models. Related Resources View Documentation JSON Schema JSON Schema-based validation is allowed with the SQL condition IS JSON and with a PL/SQL utility function. A JSON schema is a JSON document that specifies allowed properties (field names) and the corresponding allowed data types, and whether they are optional or mandatory. By default, JSON data is schemaless, providing flexibility. However, you may want to ensure that your

#### Apply a reranker to the (question, text) pairs

In [11]:
# Reranker is deployed in OCI Data Science as a Model Deployment

reranker = create_reranker(auth=api_keys_config, verbose=True)

2024-01-11 21:58:28,638 - INFO - Created OCI reranker client...
2024-01-11 21:58:28,639 - INFO - Region: eu-frankfurt-1...
2024-01-11 21:58:28,639 - INFO - Deployment id: ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaangencdyaulxbosgii6yajt2jdsrrvfbequkxt3mepz675uk3ui3q...
2024-01-11 21:58:28,640 - INFO - 


#### Integrate in the bigger RAG picture
#### OCI GenAI

In [12]:
llm_oci = create_llm(auth=api_keys_config)

In [13]:
service_context = ServiceContext.from_defaults(llm=llm_oci, embed_model=embed_model)

In [14]:
index = VectorStoreIndex.from_vector_store(
    vector_store=v_store, service_context=service_context
)

#### The entry point for the entire RAG chain

In [15]:
query_engine = index.as_query_engine(
    similarity_top_k=TOP_K,
    node_postprocessors=[reranker],
)

#### Generating the answer

In [16]:
%%time

response = query_engine.query(question)

print(f"Question: {question}")
print("")
print(response.response)
print("")

2024-01-11 21:58:33,070 - INFO - Reranking...
2024-01-11 21:58:41,175 - INFO - ...elapsed time: 8.1 sec.


Question: Write an e-mail, to a customer called Luigi, explaining what is JSON Relational Duality in Oracle Database 23c. Explain with details.

Dear Luigi,

I would like to inform you about Oracle Database 23c's JSON Relational Duality feature. This feature allows you to access and update data in your database as either JSON documents or relational tables. 

With JSON Relational Duality, you can benefit from the strengths of both data formats without having to use complex Object Relational Mapping (ORM) tools. Here's a more detailed explanation:

JSON Relational Duality introduces "Duality Views". These are fully updatable JSON views over your relational data. Even though the data is still stored in relational tables in a highly efficient normalized format, your applications can access it in the form of JSON documents. 

This approach provides great flexibility and simplicity by overcoming the challenges that developers often face when working with relational or document models. You c

#### Analyze response: documents used for the Context after addition of the reranker

In [17]:
for node in response.source_nodes:
    print(
        f"Doc. id: {node.id_}\n{node.text}\npag: {node.metadata['page_label']}\nscore: {round(node.score, 4)}\n"
    )

Doc. id: 85da167cce74f13dc9ab3e3ed9583936debe305d130e3855bf843e132d68e79e
1 Introduction Oracle Database 23c is the next long term support release of Oracle Database. Oracle Database 23c, code named “App Simple,” accelerates Oracle's mission to make it simple to develop and run all data-driven applications. It's the sum of all the features from the Oracle Database 21c innovation release plus over 300 new features and enhancements. Key focus areas include JSON, graph, microservices, and developer productivity. Note: For information about desupported features, see Oracle Database Changes, Desupports, and Deprecations. JSON Relational Duality Data can be transparently accessed and updated as either JSON documents or relational tables. Developers benefit from the strengths of both, which are simpler and more powerful than Object Relational Mapping (ORM). See JSON-Relational Duality . Operational Property Graphs in SQL Developers can now build real-time graph analysis applications against o

#### Analysis: 

If you compare with the results of the retrieval made by the Vector Store you can see that:
* some documents have been removed
* page 2-1 is higher in the list. It has more relevance to answer to the request
* all documents retained have relevance to answer to the given query